# Import the txt files from Charon3 and merge them with the data from charon4
For the EONERC data


In [19]:
version="v1_0"

import numpy as np
import pandas as pd
import glob
import matplotlib.pyplot as plt
#from datetime import date, timedelta
from collections import defaultdict
#from shutil import copy2 as copy_file
import pickle

from my_func_mvw.functions import get_abspath, read_pickle, write_pickle, read_pickle
from my_func_mvw.functions_import_my_database import merge_data_year,import_my_database_pickle

print("Some Version information of the imported packages")
print(f"pandas: {pd.__version__}")
print(f"pickle: {pickle.format_version}")
!python --version

Some Version information of the imported packages
pandas: 1.3.4
pickle: 4.0
Python 3.8.12


In [20]:
#path to working directory
#mit \[-]\ im Pfad funktioniert die get_abspath Funktion nicht
# add additional paths to list, if you exported new data from Charon4
# if you have a lot of different paths to controller files, consider exporting the whole databse from Charon4 again
# this would increase the speed of this script (having less paths to controllers)
# but the export from Charon4 takes some time
# selecting the same repository in Charon4 does not work. Doing that, every file is copied at the end of its counterpart.

############Input###################
path_to_EONERC_data=r"..\EONERC\Data"
data_save_csv=False # True False; not implementet yet
data_save_pickle=True #True False; 
# 3188 is in Alsdorf; 3195 in EONERC
controller=3195 # only 3195 is used in this notebook at the moment

path_to_charon3_EONERC=[
    r"..\EONERC\Data\unprocessed\charon3_export_as_txt"
]
######################################

In [21]:
def import_temp_to_df(path):
    """"""
    #df = pd.read_csv(path,skiprows=7,decimal=".",delimiter="\t",index_col=0,error_bad_lines=False,warn_bad_lines=False)
    df=pd.read_csv(path, skiprows=3,decimal=",",delimiter="\t",index_col=1, dtype=float, encoding="unicode_escape")
    df = df.drop(df.columns[0],axis=1)
    df=df.T
    df.index.names = ['Date']
    df.index = pd.to_datetime(df.index, infer_datetime_format=True)
    df.rename(columns=lambda x: int(float(x)), inplace=True)
    df.columns.names=["Length [m]"]
    return df

In [22]:
channels=["1","2","3","4","5","6","8","9","10","11"]
data_2020_charon3={}
for chan in channels:
    data_2020_charon3[chan] = import_temp_to_df(path_to_charon3_EONERC[0] + f"\\chan{chan}.TXT")

In [23]:
# Save data
path_to_my_database_EONERC = path_to_EONERC_data
path_to_my_database = path_to_my_database_EONERC + "\\unprocessed"

if data_save_pickle:
    save_to_pickle = path_to_my_database + "\\data_from_charon3\\all_charon3_data_pickle"
    write_pickle(save_to_pickle, data_2020_charon3)

# Merge data from Charon3 and Charon4

In [24]:
data_charon4 = import_my_database_pickle(2021,path_to_my_database + "\\data_from_charon4\\pickle", controller=3195)



In [25]:
data_2021=data_charon4
data_2020=data_2020_charon3

In [26]:
data_all={}
for channel in channels:
    data_all[channel]=pd.concat([data_charon4[channel],data_2020_charon3[channel]])
    data_all[channel]=data_all[channel].sort_index()

In [27]:
# Save data
path_to_my_database = path_to_EONERC_data

if data_save_pickle:
    #save_to_pickle = path_to_my_database + "\\all_data_pickle"
    #write_pickle(save_to_pickle, data_all)

    save_to_pickle = path_to_my_database + "\\all_data"
    write_pickle(save_to_pickle, data_all)

    # not needed to save them again at the moment
    # save_to_pickle = path_to_my_database + "\\2021_pickle"
    # write_pickle(save_to_pickle, data_2021)